In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import MinMaxScaler


In [3]:
feature_set =  'feature_set_3'
X_train = pd.read_csv( F'./data/{feature_set}/X_train_full.csv')
y_train = pd.read_csv( F'./data/{feature_set}/y_train.csv').values.ravel()

X_val = pd.read_csv( F'./data/{feature_set}/X_valid_full.csv')
y_val = pd.read_csv( F'./data/{feature_set}/y_valid.csv').values.ravel()

C:\Users\joshu\AppData\Local\Temp\ipykernel_21520\1408638984.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv( F'./data/{feature_set}/X_train_full.csv')


In [4]:
input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

In [6]:
X_train = X_train[input_features]
X_val = X_val[input_features]


In [ ]:
def scale_data(train):
	scaler = MinMaxScaler()
	scaler.fit(train)
 
	return pd.DataFrame(scaler.transform(train), index=train.index, columns=train.columns)


In [17]:
def fit_model_and_get_predictions(classifier, X_val, y_val,
                                  X_train, y_train,scale=True):

    # By default, scales input data
    # if scale:
    #     (train_df, test_df)=scaleData(train_df,test_df,input_features)
    
    # We first train the classifier using the `fit` method, and pass as arguments the input and output features
    # start_time=time.time()
    classifier.fit(X_train, y_train)
    # training_execution_time=time.time()-start_time

    # # We then get the predictions on the training and test data using the `predict_proba` method
    # # The predictions are returned as a numpy array, that provides the probability of fraud for each transaction 
    # start_time=time.time()
    predictions_test=classifier.predict_proba(X_val)[:,1]
    # prediction_execution_time=time.time()-start_time
    
    predictions_train=classifier.predict_proba(X_train)[:,1]

    # The result is returned as a dictionary containing the fitted models, 
    # and the predictions on the training and test sets
    model_and_predictions_dictionary = {'classifier': classifier,
                                        'predictions_test': predictions_test,
                                        'predictions_train': predictions_train
                                       }
    
    return model_and_predictions_dictionary

In [18]:
classifiers_dictionary={'Logistic regression':LogisticRegression(random_state=0), 
                        # 'Decision tree with depth of two':sklearn.tree.DecisionTreeClassifier(max_depth=2,random_state=0), 
                        # 'Decision tree - unlimited depth':sklearn.tree.DecisionTreeClassifier(random_state=0), 
                        'Random forest':RandomForestRegressor(random_state=0,n_jobs=-1),
                        'XGBoost':xgboost.XGBClassifier(random_state=0,n_jobs=-1),
                       }

fitted_models_and_predictions_dictionary={}

for classifier_name in classifiers_dictionary:
    
    model_and_predictions = fit_model_and_get_predictions(classifiers_dictionary[classifier_name], X_val, y_val, 
                                                                                   X_train, y_train)
    fitted_models_and_predictions_dictionary[classifier_name]=model_and_predictions

c:\Users\joshu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AttributeError: 'RandomForestRegressor' object has no attribute 'predict_proba'

In [30]:
xgboost = XGBClassifier(n_estimators = 1000,random_state=42,n_jobs=-1)
eval_set = [(X_val, y_val)]
xgboost.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, early_stopping_rounds=100, verbose=True)

[0]	validation_0-auc:0.86952
[1]	validation_0-auc:0.86987
[2]	validation_0-auc:0.86998
[3]	validation_0-auc:0.87056
[4]	validation_0-auc:0.87109
[5]	validation_0-auc:0.87107
[6]	validation_0-auc:0.87118
[7]	validation_0-auc:0.87194
[8]	validation_0-auc:0.87536
[9]	validation_0-auc:0.87509
[10]	validation_0-auc:0.87671
[11]	validation_0-auc:0.87668
[12]	validation_0-auc:0.87882
[13]	validation_0-auc:0.87894
[14]	validation_0-auc:0.88010
[15]	validation_0-auc:0.88214
[16]	validation_0-auc:0.88205
[17]	validation_0-auc:0.88293
[18]	validation_0-auc:0.88303
[19]	validation_0-auc:0.88427
[20]	validation_0-auc:0.88462
[21]	validation_0-auc:0.88456
[22]	validation_0-auc:0.88618
[23]	validation_0-auc:0.88612
[24]	validation_0-auc:0.88611
[25]	validation_0-auc:0.88620
[26]	validation_0-auc:0.88626
[27]	validation_0-auc:0.88645
[28]	validation_0-auc:0.88635
[29]	validation_0-auc:0.88641
[30]	validation_0-auc:0.88632
[31]	validation_0-auc:0.88624
[32]	validation_0-auc:0.88632
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [31]:
roc_auc_score(y_val,model_and_predictions['predictions_test'])

0.8750777864227971